In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import time
import numpy as np
import seaborn as sns

In [ ]:
#최대한 많은 공통 feature를 만들기 위해 전처리 되기 전의 IoT-23 데이터셋을 사용
iotd = pd.read_csv("iot23_not_preprocessed.csv")

In [ ]:
#결측값 등이 처리 완료된 CICIDS2017 데이터셋을 사용
cic = pd.read_csv("cicids2017_preprocessed.csv")

In [ ]:
#이종의 데이터셋을 5:5 비율로 학습하기 위해 CICIDS2017을 IoT-23 개수만큼 랜덤 추출
cicd = cic.sample(n=1444674, replace=False, axis=0)

In [ ]:
del iotd['Unnamed: 0']

In [ ]:
iotd['duration'] = iotd['duration'].str.replace('-','0')
iotd['orig_bytes'] = iotd['orig_bytes'].str.replace('-','0')
iotd['resp_bytes'] = iotd['resp_bytes'].str.replace('-','0')

In [ ]:
iotd.loc[(iotd.label == '-   Malicious   PartOfAHorizontalPortScan'), 'label'] = 'PartOfAHorizontalPortScan'
iotd.loc[(iotd.label == '(empty)   Malicious   PartOfAHorizontalPortScan'), 'label'] = 'PartOfAHorizontalPortScan'
iotd.loc[(iotd.label == '-   Malicious   Okiru'), 'label'] = 'Okiru'
iotd.loc[(iotd.label == '(empty)   Malicious   Okiru'), 'label'] = 'Okiru'
iotd.loc[(iotd.label == '-   Benign   -'), 'label'] = 'Benign'
iotd.loc[(iotd.label == '(empty)   Benign   -'), 'label'] = 'Benign'
iotd.loc[(iotd.label == '-   Malicious   DDoS'), 'label'] = 'DDoS'
iotd.loc[(iotd.label == '-   Malicious   C&C'), 'label'] = 'C&C'
iotd.loc[(iotd.label == '(empty)   Malicious   C&C'), 'label'] = 'C&C'
iotd.loc[(iotd.label == '-   Malicious   Attack'), 'label'] = 'Attack'
iotd.loc[(iotd.label == '(empty)   Malicious   Attack'), 'label'] = 'Attack'
iotd.loc[(iotd.label == '-   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
iotd.loc[(iotd.label == '(empty)   Malicious   C&C-HeartBeat'), 'label'] = 'C&C-HeartBeat'
iotd.loc[(iotd.label == '-   Malicious   C&C-FileDownload'), 'label'] = 'C&C-FileDownload'
iotd.loc[(iotd.label == '-   Malicious   C&C-Torii'), 'label'] = 'C&C-Torii'
iotd.loc[(iotd.label == '-   Malicious   C&C-HeartBeat-FileDownload'), 'label'] = 'C&C-HeartBeat-FileDownload'
iotd.loc[(iotd.label == '-   Malicious   FileDownload'), 'label'] = 'FileDownload'
iotd.loc[(iotd.label == '-   Malicious   C&C-Mirai'), 'label'] = 'C&C-Mirai'
iotd.loc[(iotd.label == '-   Malicious   Okiru-Attack'), 'label'] = 'Okiru-Attack'

In [ ]:
cicd['Label'].value_counts()

In [ ]:
#공통 feature 9개를 제외한 나머지 feature drop
iotd = iotd.drop(columns=['ts', 'uid', 'service', 'conn_state', 'history', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'local_orig', 'local_resp','missed_bytes'])

In [ ]:
#공통 feature 9개를 제외한 나머지 feature drop
cicd = cicd.drop(columns=['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Destination Port', 'Timestamp', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length', 'Bwd Header Length','Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/Up Ratio', 'Average Packet Size', 'Avg Fwd Segment Size', 'Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk', 'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk', 'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets', 'Subflow Bwd Bytes', 'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward', 'Active Mean', 'Active Std', 'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min',  'External IP' ])

In [1]:
#같은 뜻이나 이름이 다른 feature들을 rename해 매칭
iotd = iotd.rename(columns={'duration':'Flow Duration','orig_pkts':'Total Fwd Packets', 'resp_pkts':'Total Backward Packets', 'proto':'Protocol', 'orig_bytes':'Fwd Packet Length Mean' , 'resp_bytes': 'Bwd Packet Length Mean' , 'orig_ip_bytes': 'Fwd Packets/s' ,'resp_ip_bytes': 'Bwd Packets/s' ,'label': 'Label' })

NameError: name 'iotd' is not defined

In [ ]:
#같은 공격끼리 공격명 통일
iotd = iotd.replace('Benign', 'BENIGN' )
iotd = iotd.replace('C&C-HeartBeat', 'C&C' )
iotd = iotd.replace('C&C-FileDownload', 'C&C' )
iotd = iotd.replace('C&C-Torii', 'C&C' )
iotd = iotd.replace('C&C-HeartBeat-FileDownload', 'C&C' )
iotd = iotd.replace('C&C-Mirai', 'C&C' )
iotd = iotd.replace('PartOfAHorizontalPortScan', 'PortScan' )

iotd = iotd.replace('tcp', '6' )
iotd = iotd.replace('udp', '17' )
iotd = iotd.replace('icmp', '1' )

In [ ]:
iotd['Label'].value_counts()

In [ ]:
cicd['Label'].value_counts()

In [ ]:
#CICIDS2017 데이터셋 내 종류가 다양한 DoS를 모두 DoS로 통일
cicd = cicd.replace('DoS Hulk', 'DoS' )
cicd = cicd.replace('DoS GoldenEye', 'DoS' )
cicd = cicd.replace('DoS slowloris', 'DoS' )
cicd = cicd.replace('DoS Slowhttptest', 'DoS' )

In [ ]:
frames = [iotd, cicd]
df_h = pd.concat(frames)

In [ ]:
#결측값 채움
iotd.fillna(-1,inplace=True)
cicd.fillna(-1,inplace=True)
df_h.fillna(-1,inplace=True)

In [ ]:
iotd.isna().sum()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()

In [ ]:
#이종의 데이터셋으로 학습
X = df_h[['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s']]
Y = df_h['Label']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=10, test_size=0.2)

In [ ]:
#CICIDS2017로 테스트한 경우
Z = cicd[['Protocol', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Fwd Packet Length Mean', 'Bwd Packet Length Mean', 'Fwd Packets/s', 'Bwd Packets/s']]
W = cicd['Label']
Z_train, Z_test, W_train, W_test = train_test_split(Z, W, random_state=10, test_size=0.4)

In [ ]:
X_test = Z_test
Y_test = W_test

In [ ]:
start = time.time()
print('program start...')
print()

RF.fit(X_train, Y_train)  # fit() : 훈련 수행
print()
print('prediction:')
y_pred = RF.predict(X_test)  # predict() : 모델 훈련 후 예측
print(y_pred)
print()

print('Score:')
score = RF.score(X_test,Y_test)  # score() : 평가 점수 계산
print(score)

end = time.time()  # 현재 시각 구하기
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')
print("Classifiction Report :")
print(classification_report(Y_test, y_pred))